In [1]:
# testing model response
from openai import OpenAI

client = OpenAI(
    base_url="http://10.173.18.85:1234/v1",
    api_key="lm-studio"
)

response = client.chat.completions.create(
    model="llama-2-13b",
    messages=[{"role": "user", "content": "Say hello in one short sentence."}],
)

print(response.choices[0].message.content)

Hi!




In [4]:
from openai import OpenAI
from docx import Document
import pandas as pd
import json
import time

# -----------------------------
# LLM Client Setup
# -----------------------------
client = OpenAI(
    base_url="http://10.173.18.85:1234/v1",
    api_key="lm-studio"
)
model_name = "llama-2-13b"

# -----------------------------
# File Paths
# -----------------------------
INPUT_FILE = "750-tweets-without answer.xlsx"
OUTPUT_FILE = "Llama2_zero-shot.xlsx"
KEYWORDS_DOCX = "Keywords list_final.docx"

# -----------------------------
# Batch settings
# -----------------------------
BATCH_SIZE = 100
START_EXCEL_ROW = 2  # user request
START_IDX = START_EXCEL_ROW - 1  # pandas index (row2 -> idx1)

# -----------------------------
# JSON Schema for classification
# -----------------------------
classification_schema = {
    "type": "json_schema",
    "json_schema": {
        "name": "AgeismClassification",
        "schema": {
            "type": "object",
            "properties": {
                "classification": {
                    "type": "string",
                    "enum": ["Not relevant", "Ageist", "Age-positive"]
                },
                "reasoning": {
                    "type": "string",
                    "description": "Brief explanation justifying the classification decision"
                }
            },
            "required": ["classification", "reasoning"]
        },
    }
}

# -----------------------------
# Load keyword list
# -----------------------------
keywords_doc = Document(KEYWORDS_DOCX)
keywords_text = "\n".join([p.text for p in keywords_doc.paragraphs]).strip()

# -----------------------------
# Load input Excel file
# -----------------------------
df = pd.read_excel(INPUT_FILE, dtype=str)

# Ensure needed columns exist
if "classification" not in df.columns:
    df["classification"] = ""
if "reasoning" not in df.columns:
    df["reasoning"] = ""

# -----------------------------
# Prompt template
# -----------------------------
PROMPT_TEMPLATE = """
You are an expert in social psychology and research on age beliefs. 

Our goal is to create 3 lists of tweets.  To create these 3 lists, first identify all tweets that describe an older person or older people in general using Keyword List A from the attached document.

Once you have identified the tweets describing older persons, classify them into one of 3 categories: (1) ageist tweets using Keyword List B and the definition below; (2) age-positive tweets using Keyword List C and the corresponding definition below; or 3) not relevant tweets. 

The final dataset should include the following columns for each tweet: a single classification label (Ageist, Age-positive, or Not relevant) and reasoning explaining why the classification was made (e.g., “is ageist because uses age stereotype of frailty” or “is age-positive because praises older adults for resilience”).

Our definition of “ageist” tweets are tweets that are related to older persons and show an unfavorable portrayal and/or treatment of older persons.
Our definition of “age-positive” tweets are those that are related to older persons and show the favorable portrayal and/or treatment of older persons.
Our definition of “not relevant” tweets are those that are not related to older persons and/ or are related to older persons but describes them in a neutral way that is not ageist or age-positive.

When analyzing and classifying tweets, explicitly pay close attention to sarcasm, humor, contextual ambiguity, cultural references, figurative language, and nuanced sentiment. Tweets may use irony, exaggeration, slang, or culturally specific expressions that change their meaning. Do not rely solely on keywords; always interpret the intended meaning within context before classifying.

When classifying a tweet as related to older persons, please note the following rules. Please do not consider a tweet related to an older person when: 
(1) tweets describe old objects (e.g., “old car,” “old house”) unless the term refers to an old body part (e.g., “old knees” or “old brain”). 
(2) “old,” refers to “old + testament” or old refers to something that is not an older person, like "the old you," "the old we," "old time," "old adage," or "old outhouse"  
(3) “senior(s)” refers to high school or college seniors. 
(4) “pop” refers to soda, a verb, music, food (“pop tart”), or could refer to a younger person (e.g., “my pop” could mean a young “dad”). 
(5) For “elder” refers to Elder Scrolls (the video game). 
(6) For “granny,” refers to granny smith which is an apple. 
(7) For “antique” and “ancient,” describes objects rather than people. 
(8) For “nursing home,” “assisted living,” or “care home,” does not refer to older persons.
(9) the keyword “elder” is only referring to someone who is older than someone else, such as an “elder sibling,”  but is not referring to an older person.

In addition to the rules listed above, please apply the following clarification rules:
(1) Tweets that mention dementia, Alzheimer’s disease, memory loss, cognitive decline, or similar conditions may be classified as ageist, even if the tone appears sympathetic. This because the stereotype that all older persons develop dementia is a common negative age stereotype 
(2) Tweets that portray an older adult in a negative, mocking, complaining, or burdensome way (such as criticizing their cooking, speech, driving, abilities, slowness, confusion, or behavior) should be classified as ageist, even if the tweet is casual or humorous. For example, “my grandma’s cooking smells terrible” or “grandpa won’t stop repeating himself” are ageist.
(3) Tweets that express positive affection, respect, gratitude, pride, love, or admiration for an older adult should be classified as age-positive, including when someone says they miss or cherish an older relative who has passed away. This type of emotional remembrance about older persons counts as age-positive.
(4) Tweets that describe celebrating an older person, including celebrating a birthday, should be counted as age-positive.
(5) The word “retired” should only be coded as aging-related when it describes a person retiring due to age, not when it is used jokingly (e.g., “I’m retired from dating”).
(6) The term “middle-aged” does not count as older adulthood and should not be classified as aging-related.
(7) When terms like “grandma,” “grandpa,” or “old man” are used metaphorically to describe the speaker’s behavior in an ambiguous way (e.g., staying in and going to bed early), code as not relevant; However, if is clear the behavior is thought of in a negative way, code as ageist. 
(8) tweets that infantilize older persons or treat them like babies or small children should be labeled as ageist.

Also, please see the correct classifications of the following 8 tweets that could assist as you decide how to classify tweets: 
1. "Everybody grumpy today"
   - Keywords: "grumpy" (negative age stereotype)
   - Classification: Not relevant to ageist or age-positive list (as it does not specifically refer to older persons)
 
2. "Y'all clamoring for the days of the Old Testament y'all would be slaves in those days"
   - Keywords: "Old Testament" (irrelevant as it refers to the Bible, not older people)
   - Classification: Not relevant to ageist or age-positive list
 
3. "I lost my granny early in life but I cherish the moments I had with her forever"
   - Keywords: "granny"
   - Classification: Age-positive (demonstrates value and positive memory of an older person)
  
4. "WHAT?! You missed Grandparents Storytime last week? Guess what? You have a chance to catch it again on Saturday, March 21, at 10:30 a.m. Miss Tracey will sing songs, have craft time and share wonderful stories for…"
   - Keywords: "Grandparents"
   - Classification: Age-positive (promotes engagement and positive role of grandparents)
 
5. "@captaintommoore has died from #COVID19. The 100-year-old helped raise almost £33m ($57M Cdn.) for the NHS. He was knighted last year. The Army veteran won the hearts of millions by walking 100 laps in his garden before his 100th birthday last April. #Hero #Positivity #Smile"
   - Keywords: "100-year-old", "Hero", "Positivity"
   - Classification: Age-positive (highlights achievements and positive contributions of an older person)
 
6. "@Hichestan7 @SecPompeo @mikepompeo Still drinking from Trump’s septic tank? Its not good for you. The lies & bullshit eat away at your brain, accelerating dementia."
   - Keywords: "dementia" (negative age stereotype)
   - Classification: Ageist (uses dementia derogatorily)
 
7. "My old man bladder is so thankful! Not a lot more stressful than worrying if you can hold it the whole movie or not!"
   - Keywords: "old man bladder" (negative age reference)
   - Classification: Ageist (implies bladder problem due to old age in a negative way)
 
8. "Hawk Nation tune-in Sun, 4/19 on #Knightline as Coach Derek Smith (@chowan6464) discusses the recent UCF verbal commitment by Hawks speedy senior WR @davismallinger1 Live from 10-11:00 am with coach’s segment airing 10:20 am. ESPN Orlando 580 AM Hosted by @AP_Knightline https://t.co/yVmMX2jTyn"
   - Keywords: "senior"
   - Classification: Not relevant to ageist or age-positive list (senior refers to high school senior, not older person)

9.  “I pop up on ya block.”
  - Keywords: "pop"
 - Classification: Not relevant because pop used as a verb so tweet not about an older person. 

Keyword list:
{keywords_text}

Post:
{text}
"""

def classify_tweet(text: str):
    prompt_text = PROMPT_TEMPLATE.format(keywords_text=keywords_text, text=text or "")

    response = client.chat.completions.create(
        model=model_name,
        messages=[{
            "role": "user",
            "content": [{"type": "text", "text": prompt_text}]
        }],
        response_format=classification_schema,
    )

    raw = response.choices[0].message.content
    if isinstance(raw, str):
        raw = raw.strip()

    try:
        parsed = json.loads(raw) if isinstance(raw, str) else raw
        classification = parsed.get("classification", "Not relevant")
        reasoning = parsed.get("reasoning", "")
    except Exception:
        classification = "Not relevant"
        reasoning = ""

    return classification, reasoning, raw

# -----------------------------
# Batch loop: 100 per batch, start at row 2
# -----------------------------
n = len(df)
if START_IDX >= n:
    raise ValueError(f"START_EXCEL_ROW={START_EXCEL_ROW} is beyond the dataset (n={n}).")

batch_start = START_IDX
batch_num = 1

while batch_start < n:
    batch_end = min(batch_start + BATCH_SIZE, n)
    print(f"\n🚀 Batch {batch_num}: rows {batch_start+1} to {batch_end} (pandas idx {batch_start}..{batch_end-1})")

    for i in range(batch_start, batch_end):
        start_time = time.time()

        text = df.at[i, "text"] if "text" in df.columns else None
        if text is None:
            raise KeyError("Column 'text' not found. Please rename your tweet text column to 'text'.")

        classification, reasoning, raw = classify_tweet(text)

        df.at[i, "classification"] = classification
        df.at[i, "reasoning"] = reasoning

        print(f"📄 Tweet {i+1}/{n} | 🏷️ {classification} | ⏱️ {round(time.time()-start_time, 2)}s")

    # Save after each batch (safer)
    df.to_excel(OUTPUT_FILE, index=False)
    print(f"✅ Saved progress to {OUTPUT_FILE}")

    batch_start = batch_end
    batch_num += 1

print(f"\n🎉 All done. Final saved to {OUTPUT_FILE}")


🚀 Batch 1: rows 2 to 101 (pandas idx 1..100)
📄 Tweet 2/750 | 🏷️ Not relevant | ⏱️ 9.72s
📄 Tweet 3/750 | 🏷️ Ageist | ⏱️ 3.54s
📄 Tweet 4/750 | 🏷️ Not relevant | ⏱️ 4.48s
📄 Tweet 5/750 | 🏷️ Age-positive | ⏱️ 2.97s
📄 Tweet 6/750 | 🏷️ Ageist | ⏱️ 4.14s


KeyboardInterrupt: 

In [5]:
import pandas as pd

# -----------------------------
# Load files
# -----------------------------
pred_df = pd.read_excel("Llama2_zero-shot.xlsx")
ans_df  = pd.read_excel("750-tweets-answer.xlsx")

# Column names
pred_col = "classification"
aging_col = "aging related (yes/ no)"
ageist_col = "ageist (yes/ no)"
pos_col = "age positive (yes/ no)"

# -----------------------------
# Step 1: Create gold label using your rules
# -----------------------------
def create_gold_label(row):
    aging = row[aging_col].strip().upper()
    ageist = row[ageist_col].strip().upper()
    agepos = row[pos_col].strip().upper()

    if aging == "Y" and ageist == "Y":
        return "Ageist"
    elif aging == "Y" and agepos == "Y":
        return "Age-positive"
    else:
        return "Not relevant"

ans_df["gold_label"] = ans_df.apply(create_gold_label, axis=1)

# -----------------------------
# Step 2: Clean predicted label formatting
# -----------------------------
pred_df["pred_label"] = (
    pred_df[pred_col]
    .astype(str)
    .str.strip()
    .str.replace("-", "", regex=False)
    .str.title()
)

pred_df["pred_label"] = pred_df["pred_label"].replace({
    "Agepositive": "Age-positive",
    "Ageist": "Ageist",
    "Not Relevant": "Not relevant",
})

# -----------------------------
# Step 3: Add gold_label next to pred_label
# -----------------------------
pred_df["gold_label"] = ans_df["gold_label"].values

# Reorder columns so gold_label is right next to pred_label
cols = list(pred_df.columns)
pred_idx = cols.index("pred_label")
cols.insert(pred_idx + 1, cols.pop(cols.index("gold_label")))
pred_df = pred_df[cols]

# -----------------------------
# Step 4: Compare predicted vs gold
# -----------------------------
pred_df["correct"] = pred_df["pred_label"] == pred_df["gold_label"]

accuracy = pred_df["correct"].mean()

print("\n====================================")
print(f"Accuracy: {accuracy:.2%}")
print(f"Correct predictions: {pred_df['correct'].sum()} / {len(pred_df)}")
print("====================================\n")

# Save output
pred_df.to_excel("Llama2_accuracy_output.xlsx", index=False)



Accuracy: 32.53%
Correct predictions: 244 / 750



In [6]:
from docx import Document
from docx.shared import Pt

# -----------------------------
# Extract misclassified tweets
# -----------------------------

mis_df = pred_df.loc[~pred_df["correct"]].copy()
mis_df["gold_label"] = ans_df.loc[~pred_df["correct"], "gold_label"].values

print(f"Misclassified: {len(mis_df)} / {len(pred_df)}")

# -----------------------------
# Save misclassified tweets to a Word document
# -----------------------------
DOCX_FILE = "Llama2_misclassified.docx"

doc = Document()
doc.add_heading("Misclassified Tweets", level=1)
doc.add_paragraph(f"Total tweets: {len(pred_df)}")
doc.add_paragraph(f"Misclassified: {len(mis_df)}")
doc.add_paragraph(f"Accuracy: {accuracy:.2%}")

# Helpful: include columns if they exist
text_col = "text" if "text" in mis_df.columns else None

for idx, r in mis_df.reset_index(drop=False).iterrows():
    doc.add_heading(f"Case {idx+1}", level=2)

    # tweet id/index (if useful)
    if "index" in r:
        doc.add_paragraph(f"Row index: {r['index']}")

    # tweet text
    if text_col:
        doc.add_paragraph("Tweet:", style=None)
        doc.add_paragraph(str(r[text_col]))

    # labels
    doc.add_paragraph(f"Predicted: {r['pred_label']}")
    doc.add_paragraph(f"Gold: {r['gold_label']}")

    doc.add_paragraph("-" * 40)

doc.save(DOCX_FILE)
print(f"✅ Misclassified tweets saved to: {DOCX_FILE}")

# (Optional) also save the misclassified subset to Excel/CSV
mis_df.to_excel("Llama2_misclassified.xlsx", index=False)
mis_df.to_csv("Llama2_misclassified.csv", index=False)

Misclassified: 506 / 750
✅ Misclassified tweets saved to: Llama2_misclassified.docx
